<a href="https://colab.research.google.com/github/eugin0901/eugin0901/blob/main/17%2B17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas scikit-learn openpyxl pyLDAvis numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 10.0 MB/s eta 0:00:00


In [3]:
# 필요한 라이브러리 설치 및 불러오기
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import re
import pyLDAvis
import os

# pyLDAvis 시각화 저장 경로 설정
output_dir = "lda_visualizations"
os.makedirs(output_dir, exist_ok=True)

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
# 데이터 로드
data = pd.read_excel("final_datalist.xlsx", engine="openpyxl")

# 시기 분류 함수
def classify_period(date):
    if pd.isnull(date):
        return "알 수 없음"
    elif date.year <= 2014:
        return "1기"
    elif date.year <= 2019:
        return "2기"
    else:
        return "3기"

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
# 날짜 열을 datetime 형식으로 변환
data['일자'] = pd.to_datetime(data['일자'], format='%Y%m%d', errors='coerce')

# 시기 분류 열 추가
data['시기'] = data['일자'].apply(classify_period)

# 통합 텍스트 생성
data['통합 텍스트'] = data['키워드'].fillna('') + ' ' + data['특성추출(가중치순 상위 50개)'].fillna('')

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
# 전처리 함수 정의
def preprocess_text(text):
    try:
        # 한글만 남기고 정규화
        text = re.sub(r'[^가-힣\s,]', '', text)  # 쉼표(,)는 유지
        # 쉼표를 기준으로 분리
        words = text.split(',')  # 쉼표 기준으로 분리
        # 두 글자 이상의 단어만 추출
        words = [word.strip() for word in words if len(word.strip()) > 1]  # 공백 제거 후 길이 체크
        return ' '.join(words)  # 다시 문자열로 반환 (단어들을 공백으로 연결)
    except Exception as e:
        print(f"전처리 중 오류 발생: {e}, 입력 데이터: {text}")
        return ''

# 전처리된 텍스트 생성
data['전처리된 텍스트'] = data['통합 텍스트'].apply(preprocess_text)

# 시기별 데이터 분할
periods = {
    "1기": data[data['시기'] == "1기"],
    "2기": data[data['시기'] == "2기"],
    "3기": data[data['시기'] == "3기"]
}

# 각 시기별 데이터 크기 확인
for period, df in periods.items():
    print(f"{period} 데이터 개수: {len(df)}")

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


1기 데이터 개수: 117
2기 데이터 개수: 364
3기 데이터 개수: 887


In [28]:
# 한국어 불용어 리스트 (필요에 따라 확장 가능)
korean_stopwords = [
    "이", "그", "저", "것", "수", "등", "들", "및", "에서", "그리고", "하지만", "또한", "를", "은", "는", "에", "의", "와", "과", "로", "으로", "하다"
]

# LDA 모델링 함수 수정
def lda_topic_modeling(texts, n_topics=3, n_words=10):
    """
    LDA 토픽 모델링 함수
    - texts: 전처리된 텍스트 리스트
    - n_topics: 추출할 토픽 수
    - n_words: 각 토픽에서 출력할 단어 수
    """
    vectorizer = CountVectorizer(
        max_df=0.95,
        min_df=2,
        stop_words=korean_stopwords  # 한국어 불용어 적용
    )
    dtm = vectorizer.fit_transform(texts)  # 문서-단어 행렬 생성

    lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)  # LDA 모델
    lda.fit(dtm)

    # 토픽별 주요 단어 출력
    words = vectorizer.get_feature_names_out()
    topics = {}
    for topic_idx, topic in enumerate(lda.components_):
        topics[f"Topic {topic_idx+1}"] = [words[i] for i in topic.argsort()[:-n_words - 1:-1]]

    return topics, lda, dtm, vectorizer




/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [29]:
!pip install konlpy

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [30]:
# 시기별 LDA 실행
lda_results = {}
for period, df in periods.items():
    print(f"\n===== {period} =====")
    topics, lda_model, dtm, vectorizer = lda_topic_modeling(df['전처리된 텍스트'], n_topics=3, n_words=10)
    lda_results[period] = (topics, lda_model, dtm, vectorizer)
    for topic, words in topics.items():
        print(f"{topic}: {', '.join(words)}")

# 토픽 간 유사도 비교 함수
def compare_topics(topic_words_1, topic_words_2):
    """
    두 시기의 토픽 단어를 비교하여 유사도를 계산
    - topic_words_1, topic_words_2: 각 시기의 토픽 단어 리스트
    """
    common_words = set(topic_words_1).intersection(set(topic_words_2))
    return len(common_words) / len(set(topic_words_1).union(set(topic_words_2)))

# Hot-Cold 트렌드 분석
print("\n===== Hot-Cold 트렌드 분석 =====")
previous_period = None
for period, (topics, _, _, _) in lda_results.items():
    if previous_period:
        print(f"\n{previous_period} -> {period} 변화:")
        for topic_idx, words in enumerate(topics.values()):
            similarities = [
                compare_topics(words, prev_words)
                for prev_words in lda_results[previous_period][0].values()
            ]
            max_similarity = max(similarities)
            print(f"  Topic {topic_idx+1} 유사도: {max_similarity:.2f} (이전 시기와 비교)")
    previous_period = period

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



===== 1기 =====
Topic 1: 치과, 디지털, 장비, 바텍, 제품, 대표, 시장, 기업, 기기, 의료
Topic 2: 치과, 산업, 임플란트, 사업, 의료, 디오, 기업, 디지털, 시장, 지원
Topic 3: 임플란트, 치아, 치료, 치과, 환자, 시술, 병원, 수술, 원장, 가능

===== 2기 =====
Topic 1: 시장, 매출, 중국, 덴티움, 치의학, 매출액, 디지털, 치과, 대비, 레이
Topic 2: 치과, 디지털, 임플란트, 기업, 시장, 제품, 오스템임플란트, 대표, 의료, 덴티스트리
Topic 3: 임플란트, 치아, 환자, 치료, 치과, 디지털, 수술, 시술, 제작, 교정

===== 3기 =====
Topic 1: 투자, 기업, 인수, 메디트, 시장, 글로벌, 펀드, 사업, 산업, 지원
Topic 2: 시장, 중국, 매출, 오스템임플란트, 레이, 실적, 성장, 대비, 매출액, 임플란트
Topic 3: 치과, 임플란트, 디지털, 치료, 치아, 환자, 수술, 교정, 구강, 의료

===== Hot-Cold 트렌드 분석 =====

1기 -> 2기 변화:
  Topic 1 유사도: 0.18 (이전 시기와 비교)
  Topic 2 유사도: 0.54 (이전 시기와 비교)
  Topic 3 유사도: 0.54 (이전 시기와 비교)

2기 -> 3기 변화:
  Topic 1 유사도: 0.11 (이전 시기와 비교)
  Topic 2 유사도: 0.43 (이전 시기와 비교)
  Topic 3 유사도: 0.67 (이전 시기와 비교)
